## Reading 2021 Data

In [1]:
import pandas as pd
import numpy as np

In [96]:
headers_name = ["Stn_code", "Location", "State_Name", "Temp_min", "Temp_max", "pH_min", "pH_max",
                "conduc_min", "conduc_max", "bod_min", "bod_max", "nitrate_min", "nitrate_max",
                "Fcoli_min", 'Fcoli_max', "Tcoli_min", "Tcoli_max", "Tds_min", "Tds_max", "F_min",
                "F_max", "Ar_min", "Ar_max"]

In [97]:
data = pd.read_excel("NWMP_DATA_2021.xlsx", engine="openpyxl", skiprows=[0], header=[1])
data.columns.values[0:data.shape[1]] = headers_name
print(data.shape)
data.head(3)

(1090, 23)


,Stn_code,Location,State_Name,Temp_min,Temp_max,pH_min,pH_max,conduc_min,conduc_max,bod_min,...,Fcoli_min,Fcoli_max,Tcoli_min,Tcoli_max,Tds_min,Tds_max,F_min,F_max,Ar_min,Ar_max
0,3092.0,"BORE WELL AT ARINAMA AKKIVALASA, SRIKAKULAM",ANDHRA PRADESH,28,31,7.4,8.1,712,1139,2,...,3,4,64,75,492,754,1,1.1,0.001,0.001
1,4360.0,"BORE WELL AT IDA,\nRAMANAYYAPETA, KKAINADA",ANDHRA PRADESH,27,29,7.7,7.8,990,1280,1.8,...,2,7,75,93,660,868,0.4,0.4,0.007,0.009
2,3091.0,"BORE WELL AT KAPULUPPADA\nDUMPSITE, VISHAKHAPA...",ANDHRA PRADESH,26,26,7.0,7.1,3340,4260,2.2,...,4,7,93,93,2320,2832,0.8,1.1,0.001,0.001


In [40]:
data.drop(['Stn_code', "Location", "State_Name"], axis=1, inplace=True)
data.replace({'-':np.nan}, inplace=True)

In [66]:
data_N= data.dropna(subset=['nitrate_min'])
data_F = data.dropna(subset=['F_min'])
data_Ar = data.dropna(subset=['Ar_min'])

## Reading 2020 Data

In [68]:
import pandas as pd
import numpy as np

In [88]:
headers_name = ["Stn_code", "Location", "State_Name", "Temp_min", "Temp_max", "pH_min", "pH_max",
                "conduc_min", "conduc_max", "bod_min", "bod_max", "nitrate_min", "nitrate_max",
                "Fcoli_min", 'Fcoli_max', "Tcoli_min", "Tcoli_max", "FS_min", "FS_max", "Tds_min",
                "Tds_max", "F_min", "F_max", "Ar_min", "Ar_max"]

In [94]:
data = pd.read_excel("NWMP_DATA_2020.xlsx", engine="openpyxl", sheet_name="Table 1", header=[1])
data.columns.values[0:data.shape[1]] = headers_name

for table in range(2, 30, 1):
  if table==14: continue
  data_temp = pd.read_excel("NWMP_DATA_2020.xlsx", engine="openpyxl", sheet_name=f"Table {table}", header=[1])
  data_temp.columns.values[0:data_temp.shape[1]] = headers_name
  data = pd.concat([data, data_temp], axis=0)

data.shape


(942, 25)

In [95]:
data.to_csv("Data_2020_combined.csv")

# Analysis Post processing

In [2]:
import pandas as pd
import numpy as np

In [15]:
data21 = pd.read_csv("data\Data_2021_combined.csv")
data20 = pd.read_csv("data\Data_2020_combined.csv")
#data21.head(6)

In [16]:
data21 = data21.replace({'-':-1})
data21.dropna(how='all', inplace=True)
#data21 = data21.apply(pd.to_numeric)
data21.drop(data21[data21['State_Name'].isna()].index, axis=0, inplace=True)
data_num = data21.drop(['Stn_code', 'Location', 'State_Name'], axis=1)
data_num = data_num.apply(pd.to_numeric)
#data_num.info()

In [37]:
(data_num['nitrate_max']==-1).value_counts()

nitrate_max
False    965
True     117
Name: count, dtype: int64

In [17]:
data20 = data20.replace('\n','', regex=True)
data20 = data20.replace({'-':-1, 'BDL':0})
data20.dropna(how='all', inplace=True)
#data20 = data20.apply(pd.to_numeric)
data20.drop(data20[data20['State_Name'].isna()].index, axis=0, inplace=True)
data_num1 = data20.drop(['Stn_code', 'Location', 'State_Name', 'FS_min', 'FS_max'], axis=1)
data_num1 = data_num1.apply(pd.to_numeric)
#data_num1.info()

In [29]:
#final_data = pd.concat([data_num, data_num1], axis=0)
#final_data.shape
data = pd.read_csv("data\dataf.csv")
data.drop(['Unnamed: 0'], axis=1, inplace=True)
data.columns

Index(['Temp_min', 'Temp_max', 'pH_min', 'pH_max', 'conduc_min', 'conduc_max',
       'bod_min', 'bod_max', 'nitrate_min', 'nitrate_max', 'Fcoli_min',
       'Fcoli_max', 'Tcoli_min', 'Tcoli_max', 'Tds_min', 'Tds_max', 'F_min',
       'F_max', 'Ar_min', 'Ar_max'],
      dtype='object')

In [41]:
n_data = data.drop(['F_min', 'F_max', 'Ar_min', 'Ar_max'], axis = 1)
n_data = n_data[n_data['nitrate_max']!=-1]
len(n_data)

1771

ML Testing

In [42]:
y_max= n_data.pop('nitrate_max')
y_min = n_data.pop('nitrate_min')
X = n_data

In [57]:
from sklearn.model_selection import train_test_split, GridSearchCV
X_train, X_test, y_train, y_test = train_test_split(X, y_max, test_size=0.2)

In [75]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
parameters = {'n_estimators':[90, 100, 110, 120],
              'max_depth':[4, 5, 6, 7, 8, 9, 10],
              'min_samples_split':[2, 3],
              'min_samples_leaf':[3, 4, 5, 6, 7]
              }
rf = RandomForestRegressor(random_state=0)
clf = GridSearchCV(rf, param_grid=parameters, refit=True, cv=5, n_jobs=-1, verbose=1)
clf.fit(X_train, y_train)

Fitting 5 folds for each of 280 candidates, totalling 1400 fits


GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=0), n_jobs=-1,
             param_grid={'max_depth': [4, 5, 6, 7, 8, 9, 10],
                         'min_samples_leaf': [3, 4, 5, 6, 7],
                         'min_samples_split': [2, 3],
                         'n_estimators': [90, 100, 110, 120]},
             verbose=1)

In [80]:
gbr = GradientBoostingRegressor(random_state=0)
gbr.fit(X_train, y_train)

GradientBoostingRegressor(random_state=0)

In [79]:
rf.feature_importances_

array([0.05436   , 0.05123633, 0.05272872, 0.15436133, 0.24713751,
       0.11813711, 0.02140547, 0.02583037, 0.0064526 , 0.01323187,
       0.04449371, 0.05798453, 0.07450034, 0.07814011])

In [81]:
y_pred = gbr.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rsq = r2_score(y_test, y_pred)
print(mse, rsq)

62.945792886725904 0.2916015931419277
